In [1]:
import os

import numpy as np
import pandas as pd
from SALib.sample import latin
from joblib import Parallel, delayed


# Setup

In [15]:
# set random seed for reproducibility
seed_value = 123

# directory where the data is stored
data_dir = "/Users/d3y010/projects/statemod/data/inputs/cm2015_StateMod/StateMod"

# template file as a source for modification
template_file = os.path.join(data_dir, "cm2015B.ddm")

# directory to write modified files to
output_dir = "/Users/d3y010/Desktop"

# scenario name
scenario = "test"

# character indicating row is a comment
comment = "#"

# dictionary to hold values for each field
d = {"yr": [], 
     "id": [], 
     "oct": [], 
     "nov": [], 
     "dec": [], 
     "jan": [], 
     "feb": [], 
     "mar": [], 
     "apr": [], 
     "may": [],
     "jun": [],
     "jul": [],
     "aug": [],
     "sep": [],
     "total": []}

# define the column widths for the output file
column_widths = {"yr": 4, 
                 "id": 9, 
                 "oct": 10, 
                 "nov": 7, 
                 "dec": 7, 
                 "jan": 7, 
                 "feb": 7, 
                 "mar": 7, 
                 "apr": 7, 
                 "may": 7,
                 "jun": 7,
                 "jul": 7,
                 "aug": 7,
                 "sep": 7,
                 "total": 9}

# list of columns to process
column_list = ["yr", "id", "oct", "nov", "dec", "jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "total"]

# list of value columns that may be modified
value_columns = ["oct", "nov", "dec", "jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "total"]




# Test single file first

## ingest file

In [6]:
%%time

# empty string to hold header data
header = ""

capture = False
with open(template_file) as template:
    
    for idx, line in enumerate(template):
        
        if capture:
            
            # strip newline and split on spaces
            value_list = line.strip().split()

            # comprehension only used to build dict
            x = [d[column_list[idx]].append(i) for idx, i in enumerate(value_list)]

        else:
            
            # store any header and preliminary lines to use in restoration
            header += line
            
            # passes uncommented date range line before data start and all commented lines in header
            if line[0] != comment:
                capture = True
                
# convert dictionary to a pandas data frame  
df = pd.DataFrame(d)

# convert value column types to float
df[value_columns] = df[value_columns].astype(np.float64)

df


CPU times: user 397 ms, sys: 26.7 ms, total: 424 ms
Wall time: 424 ms


,yr,id,oct,nov,dec,jan,feb,mar,apr,may,jun,jul,aug,sep,total
0,1909,3600507,11.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,251.0,138.0,52.0,30.0,554.0
1,1909,3600603,39.0,0.0,0.0,0.0,0.0,0.0,3.0,64.0,304.0,263.0,251.0,80.0,1004.0
2,1909,3600606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1909,3600642,55.0,0.0,0.0,0.0,0.0,0.0,7.0,172.0,940.0,818.0,200.0,75.0,2267.0
4,1909,3600645,156.0,2.0,0.0,0.0,0.0,0.0,38.0,458.0,1590.0,1750.0,540.0,217.0,4751.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46510,2013,72_AMC001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46511,2013,72_GJMun,556.0,430.0,441.0,424.0,394.0,420.0,450.0,570.0,763.0,844.0,775.0,592.0,6659.0
46512,2013,72_UWCD,1040.0,780.0,780.0,780.0,780.0,910.0,1040.0,1300.0,1430.0,1560.0,1430.0,1170.0,13000.0
46513,2013,ChevDem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.describe()

,oct,nov,dec,jan,feb,mar,apr,may,jun,jul,aug,sep,total
count,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,46515.000000,4.651500e+04
mean,10299.688488,9832.163539,9826.958035,9817.274170,9788.435064,9858.103665,10073.946942,10751.922541,11206.293153,10954.417844,10678.840396,10497.972224,1.235860e+05
std,94755.731681,94747.446834,94757.090851,94754.772597,94746.983441,94757.056342,94749.590124,94732.058305,94706.586374,94726.609069,94746.623071,94746.100641,1.136684e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,128.000000,273.000000,228.000000,112.000000,83.000000,1.145500e+03
50%,117.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.000000,362.000000,615.000000,532.000000,340.000000,252.000000,2.526000e+03
75%,357.000000,27.000000,9.000000,5.000000,4.000000,19.000000,136.000000,892.000000,1438.000000,1120.500000,795.000000,592.000000,5.917000e+03
max,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,1.199999e+07


## make modification

In [8]:
%%time

# ids that have been identified as municipal
municipal_ids = ["3600603", "72_GJMun", "3600642"]

municial_multiplier = 0.4

# modify value columns associated with municipal ids
df[value_columns] = (df[value_columns] * municial_multiplier).where(df["id"].isin(municipal_ids), df[value_columns])

# apply precision adjustment function to match statemods format
df[value_columns] = df[value_columns].apply(np.around)

# convert all fields to str type
df = df.astype(str)

# if adjusting precision, remove trailing 0
df[value_columns] = df[value_columns].apply(lambda x: x.str[:-1])

df


CPU times: user 405 ms, sys: 19.4 ms, total: 424 ms
Wall time: 424 ms


,yr,id,oct,nov,dec,jan,feb,mar,apr,may,jun,jul,aug,sep,total
0,1909,3600507,11.,0.,0.,0.,0.,0.,0.,72.,251.,138.,52.,30.,554.
1,1909,3600603,16.,0.,0.,0.,0.,0.,1.,26.,122.,105.,100.,32.,402.
2,1909,3600606,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
3,1909,3600642,22.,0.,0.,0.,0.,0.,3.,69.,376.,327.,80.,30.,907.
4,1909,3600645,156.,2.,0.,0.,0.,0.,38.,458.,1590.,1750.,540.,217.,4751.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46510,2013,72_AMC001,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
46511,2013,72_GJMun,222.,172.,176.,170.,158.,168.,180.,228.,305.,338.,310.,237.,2664.
46512,2013,72_UWCD,1040.,780.,780.,780.,780.,910.,1040.,1300.,1430.,1560.,1430.,1170.,13000.
46513,2013,ChevDem,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.


## build new file

In [9]:
%%time

template_basename = os.path.basename(template_file)
template_name_parts = os.path.splitext(template_basename)

output_file = os.path.join(output_dir, f"{template_name_parts[0]}_{scenario}{template_name_parts[-1]}")

formatters={'id':'{{:<{}s}}'.format(df['id'].str.len().max()).format}

with open(output_file, "w") as out:
    
    # write header
    out.write(header)

    # write altered content
    df.to_string(buf=out,
                 col_space=column_widths,
                 header=False,
                 index=False,
                 formatters={'id':'{{:<{}s}}'.format(df['id'].str.len().max()).format})


CPU times: user 2.33 s, sys: 30.1 ms, total: 2.36 s
Wall time: 2.36 s


# Test multifile with LHS draws

## identify ids for each category


In [16]:
municipal_ids = ["3600603", "72_GJMun", "3600642"]
standard_ids = ["3600649_D", "3600868_D"]
alternative_ids = ["36_ADC017", "36_KeyMun"]


## generate lhs sample

In [17]:
# build our problem
problem = {
    'num_vars': 3,
    'names': ['municipal', 'standard', 'alternative'],
    'bounds': [[-1.0, 1.0],
               [-1.0, 1.0],
               [-1.0, 1.0]]
}

# generate 4 files
n_samples = 4

# generate our sample so we can test
lhs = latin.sample(problem, n_samples, seed_value)

lhs


array([[-0.65176541, -0.14026552,  0.86452485],
       [-0.22434262,  0.34241487,  0.24046595],
       [ 0.4903821 ,  0.67158901, -0.28844677],
       [ 0.69605876, -0.85693033, -0.88657427]])

## workhorse function combining the steps above

In [18]:
def prep_data(template_file, column_list, value_columns, comment="#"):
    """Ingest statemod ddm template file and format into a data frame."""
    
    # dictionary to hold values for each field
    d = {"yr": [], 
         "id": [], 
         "oct": [], 
         "nov": [], 
         "dec": [], 
         "jan": [], 
         "feb": [], 
         "mar": [], 
         "apr": [], 
         "may": [],
         "jun": [],
         "jul": [],
         "aug": [],
         "sep": [],
         "total": []}
    
    # empty string to hold header data
    header = ""

    capture = False
    with open(template_file) as template:

        for idx, line in enumerate(template):

            if capture:

                # strip newline and split on spaces
                value_list = line.strip().split()

                # comprehension only used to build dict
                x = [d[column_list[idx]].append(i) for idx, i in enumerate(value_list)]

            else:

                # store any header and preliminary lines to use in restoration
                header += line

                # passes uncommented date range line before data start and all commented lines in header
                if line[0] != comment:
                    capture = True
                    
    # convert dictionary to a pandas data frame  
    df = pd.DataFrame(d)

    # convert value column types to float
    df[value_columns] = df[value_columns].astype(np.float64)
                    
    return df, header


def apply_adjustment(data_df, value_columns, target_ids, factor):
    """Apply adjustment to template file values for target ids using a sample factor."""

    return (data_df[value_columns] * factor).where(data_df["id"].isin(target_ids), data_df[value_columns])


def workhorse(municipal_ids, 
              standard_ids, 
              alternative_ids, 
              sample, 
              sample_id, 
              output_dir, 
              column_widths, 
              data_df, 
              header,
              adjust_precision=True,
              precision_function=np.around):
    
    # break out values from sample
    municipal_factor = sample[problem["names"].index("municipal")]
    standard_factor = sample[problem["names"].index("standard")]
    alternative_factor = sample[problem["names"].index("alternative")]
    
    # copy template data frame for alteration 
    df = data_df.copy()

    # modify value columns associated with municipal, standard, and alternative ids based on the lhs draw
    df[value_columns] = apply_adjustment(df, value_columns, municipal_ids, municipal_factor)
    df[value_columns] = apply_adjustment(df, value_columns, standard_ids, standard_factor)
    df[value_columns] = apply_adjustment(df, value_columns, alternative_ids, alternative_factor)
    
    if adjust_precision:

        # apply precision adjustment function to match statemods format (e.g., 94.7 to 94.)
        df[value_columns] = df[value_columns].apply(precision_function)

    # convert all fields to str type
    df = df.astype(str)
    
    if adjust_precision:

        # if adjusting precision, remove trailing 0
        df[value_columns] = df[value_columns].apply(lambda x: x.str[:-1])
    
    # construct output file name
    template_basename = os.path.basename(template_file)
    template_name_parts = os.path.splitext(template_basename)
    output_file = os.path.join(output_dir, f"{template_name_parts[0]}_scenario-{scenario}_sample-{sample_id}{template_name_parts[-1]}")

    # write output file
    with open(output_file, "w") as out:

        # write header
        out.write(header)

        # write altered content
        df.to_string(buf=out,
                     col_space=column_widths,
                     header=False,
                     index=False,
                     formatters={'id':'{{:<{}s}}'.format(df['id'].str.len().max()).format})



## generate the new files in parallel

In [19]:
%%time

# prepare data
df, header = prep_data(template_file, column_list, value_columns, comment=comment)


CPU times: user 648 ms, sys: 25.8 ms, total: 674 ms
Wall time: 673 ms


In [24]:
%%time

results = Parallel(n_jobs=-1, backend="loky")(delayed(workhorse)(municipal_ids, standard_ids, alternative_ids, 
                                                       sample, sample_id, output_dir, column_widths, df, header) 
                                                       for sample_id, sample in enumerate(lhs))



CPU times: user 100 ms, sys: 21.9 ms, total: 122 ms
Wall time: 4.53 s
